## Satellite Navigation Assignment

**Name:** Stefan Manek

The goal of this assignment was to establish the position and clock offset of a GPS receiver when given the positions of 4 satellites, as well as the times of flight of the signals to the receiver. Converting the positions to Cartesian coordinates, there are four unknowns: $(x,y,z,b)$ where $b$ is the receiver clock offset.

Using the equations: 

$f_k(x,y,z,b) = (x-x_k)^2 + (y-y_k)^2 + (z-z_k)^2 - (d_k-b)^2 = 0$

where $k$ denotes the satellite number (1-4) and $d_k$ is the distance from the $k^{th}$ satellite to the receiver, equivalent to the time of flight times the speed of light c, we may use Newton's method to iterate and solve for the receiver position.

In [1]:
#Importing libraries
import numpy as np

Firstly, the parameters required to implement Newton's method were defined. The satellite positions in Geocentric spherical coordinates were saved as a text file when running the Receiver programme, and are read into Python from file. They are displayed in the cell below. The speed of light is defined in km/s, and a constant altitude for each satellite was assumed to be 20200km and used to define each satellite radius.

In [2]:
#Defining constants

#Speed of light in km/s
c = 299792458/1000

#Satellite radius from centre of earth:
sat_altitude = 20200
earth_radius = 6371
sat_radius = sat_altitude + earth_radius


def read_txt_file(filename):
    """Simple function to read satellite positions from text file"""
    file = open(filename, 'r')
    lines = file.readlines()
    data = []
    for line in lines:
        sat_data = line.strip().split()
        sat_data = [float(coord) for coord in sat_data]
        data.append(sat_data)
    file.close()
    return data
    
satellite_positions = read_txt_file('gps_data.txt')

#Displaying positions:
satellite_positions

[[45.631564848884736, 20.84843421301266, 0.070251801121244],
 [31.422545402507552, -2.941191209918443, 0.070479987748836],
 [43.15593823471849, -63.03374253820336, 0.074068278368025],
 [83.68072526002337, -24.123962092271057, 0.072629494377581]]

Next, functions for transforming positions between Cartesian and Spherical coordinate systems were defined using standard relations. The satellite positions were then transformed to Cartesians and saved as numpy arrays. These results are shown below.

In [3]:
def cartesian_transform(lat, long, tof, radius):
    """Converts coordinates in global spherical to carteisan coordinates, and
    converts time-of-flight to pseudorange"""
    lat, long = np.radians(lat), np.radians(long)
    x = radius * np.cos(lat) * np.cos(long)
    y = radius * np.cos(lat) * np.sin(long)
    z = radius * np.sin(lat)
    pseudorange = (tof * c)
    return [x, y, z, pseudorange]

def spherical_transform(x ,y, z):
    """Converts from global cartesian coordinates to spherical
    using standard transformations."""
    R = np.sqrt(x**2 + y**2 + z**2)
    lat = np.arcsin(z / R)
    long = np.arctan(y/x)
    return [np.degrees(lat), np.degrees(long), R]

def sat_coords_vector(positions):
    """Returns satellite positions in cartesian coordinates, assuming constant
    satellite radius defined above."""
    sat_vector = []
    for sat in positions:
        sat_vector.append(np.array(cartesian_transform(*sat, sat_radius)))
    return sat_vector

sat_positions = sat_coords_vector(satellite_positions)
sat_positions

[array([17363.75140643,  6612.6720435 , 18994.49253181, 21060.96013706]),
 array([22644.3808991 , -1163.43677329, 13852.67012424, 21129.36876703]),
 array([  8789.71027666, -17275.92196577,  18174.20024334,  22205.11123178]),
 array([ 2669.20703675, -1195.33393751, 26409.55417217, 21773.77464275])]

Functions to calculate the individual vectors required for the prediction algorithm were then defined. As a single Newton iteration is defined as follows:

$U_{n+1} = U_n - J^{-1}(U_n)F(U_n)$

where $U_n$ is a vector consisting of the unknown parameters $(x,y,z,b)^T$ at step $n$, $F$ is the vector of functions  $(f_1,f_2,f_3,f_4)^T$, while $J$ denotes a 4x4 Jacobian matrix, with each element at row $i$, column $j$ defined as: $\textit{J}_{i,j} = \frac{\partial f_i}{\partial u_j}$

As mentioned above, the functions within F are defined as follows:

$f_k(x,y,z,b) = (x-x_k)^2 + (y-y_k)^2 + (z-z_k)^2 - (d_k-b)^2 = 0 (k = 1,2,3,4)$

Therefore, it was noticed that each row of the Jacobian $J$ would be equivalent to:

$+(2u - 2u_k)$ for $u \equiv x,y,z$ and $-(2u - 2d_k)$ for $u \equiv b$

Hence a function was written to calculate the Jacobian based on these relations.

In [4]:
def f1(x_k, y_k, z_k, d_k):
    """Function that returns the function f_k when the
    satellite positions (x_k, y_k, z_k) are given as inputs"""
    def new_func(x, y, z, b):
        return (x-x_k)**2 + (y-y_k)**2 + (z-z_k)**2 - (d_k-b)**2
    return new_func

def get_f_vector(sat_positions):
    f_vector = []
    for sat in sat_positions:
        f_vector.append(f1(*sat))
    return f_vector

def get_f_array(U_n, f_vector):
    """Inputs: Vector of unknowns U_n and vector of functions f_1,...,f_4
    
    Outputs: Vector of function values of unknowns F(U_n)"""
    f_array = []
    for func in f_vector:
        f_array.append(func(*U_n))
    return np.array(f_array)


def deriv(x, x_k):
    """Simple function to calculate the dervative value for the Jacobian elements"""
    return (2*x - 2*x_k)


def calculate_J(U_n, sat_positions):
    """Function to calculate Jacobian matrix given the unknowns vector
    and positions of the satellite in Cartesian coordinates"""
    J = []
    for sat in sat_positions:
        j_row = deriv(U_n, sat)
        #As last element of each row is equivalent to -2(b-d_k):
        j_row[-1] = j_row[-1]*(-1)
        J.append(j_row)
    return J

Having defined the individual elements for Newton's method, these were combined to repeat the process over a certain number of iterations to acquire an accurate solution. The expression $J^{-1}(U_n)F(U_n)$ was acquired by solving the Matrix equation $JX = F$ for $X$ using numpy.

In [5]:
def get_next_step(U_n, F_vector, satellite_positions):
    """Function to acquire the next iteration of Newton's method, i.e. to find U_(n+1)"""
    J = calculate_J(U_n, satellite_positions)
    F = get_f_array(U_n, F_vector)
    J_1F = np.linalg.solve(J, F)
    U_n = U_n - J_1F
    return U_n, F

def convert_coords(U_n):
    """Input: Vector of guesses for unknown parameters in cartesian coordiantes
    
    Output: Latitude, longitude and elevation of location as well as
    clock offset of receiver in microseconds"""
    U_n_spherical = spherical_transform(*U_n[:3])
    lat, long = U_n_spherical[0:2]
    #elevation in metres
    elevation = (U_n_spherical[-1] - earth_radius)*1000
    #Offset in μs
    clock_offset = U_n[-1]/(c*10e-6)
    return lat, long, elevation, clock_offset
    
def display_results(U_n):
    """Simple function to print results to screen in appropriate format"""
    
    lat, long, elevation, clock_offset = convert_coords(U_n)
    print('Calculated latitude and longitude: %s, %s degrees' % (lat,long))
    print('Calculated elevation: %s m' % elevation)
    print('Calculated receiver clock offset: %s microseconds' % clock_offset)

    
def Newton_algorithm(U_n, satellite_positions, iterations=15, criterion=3e-7):
    """Function to implement Newton's method using previously defined functions.
    A convergence criterion was set to 3e-7, and the algorithm stops when the
    sum of the functions vector is below this threshold"""
    F_vector = get_f_vector(satellite_positions)
    i = 1
    while i < iterations:
        U_n, F = get_next_step(U_n, F_vector, satellite_positions)
        tolerance = np.sum(abs(F))
        if tolerance < criterion:
            print('Convergence after %s iterations' % str(i))
            display_results(U_n)
            return U_n
        else:
            i +=1
    print('Convergence not reached after %s iterations \n' % str(i))
    display_results(U_n)
    return U_n

In [6]:
U_0 = np.array([0,0,0,0])

solution = Newton_algorithm(U_0, sat_positions)

Convergence after 7 iterations
Calculated latitude and longitude: 52.367660739150935, -7.942669138458443 degrees
Calculated elevation: 61.34894579554384 m
Calculated receiver clock offset: 122.37854508042572 microseconds


As is shown above, the solution reached convergence after 7 iterations of Newton's method for a very crude initial guess of the Earth's centre, giving a location that was near Clonmell in County Tipperary as shown below:

![Position](GPS_position.png "Position derived using Newton's method")

To experiment with how much of an effect the initial guess has on this method, the centre of Ireland (roughly Athlone town) was found in spherical coordiantes using Google maps and converted to Cartesian coordinates with an assumed receiver offset of 0. As it is given in the assignment brief that the location is in Ireland, this is realistically the best initial guess we can give.

In [7]:
centre_of_ireland = [53.42640419715147, -7.926008754164638]

centre_of_ireland_cartesian = np.array(cartesian_transform(*centre_of_ireland, 0, earth_radius))

new_solution = Newton_algorithm(centre_of_ireland_cartesian, sat_positions)

Convergence after 4 iterations
Calculated latitude and longitude: 52.36766073915084, -7.942669138458065 degrees
Calculated elevation: 61.348945659119636 m
Calculated receiver clock offset: 122.37854508038838 microseconds


As shown above, this gives the exact same position with 3 fewer iterations. Therefore, the initial guess does have a demonstratable effect on convergence, however, 7 iterations is still very efficient, especially for such a basic initial guess.